In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import geopandas as gpd

# Random Coordinate in Europe

In [ ]:
coastlines = gpd.read_file('ne_10m_coastline/ne_10m_coastline.shp')

In [ ]:
coastlines.plot()

In [ ]:
coastlines

In [ ]:
# read in the country borders from Natural Earth
countries = gpd.read_file('ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')

In [ ]:
countries

In [ ]:
countries.columns

In [ ]:
countries['SUBREGION'].unique()

In [ ]:
countries_europe = countries[countries['CONTINENT']=='Europe']
countries_europe = countries_europe.drop(countries_europe[countries_europe['NAME_EN']=='Russia'].index);

In [ ]:
countries_europe.plot(column='MAPCOLOR7')

In [ ]:
countries_europe[['NAME_LONG','POP_EST']]

In [ ]:
countries_europe.columns

In [ ]:
countries_europe['GEOUNIT'].unique()

In [ ]:
countries[['SUBUNIT','NAME_EN']]

In [ ]:
countries_europe['coords'] = countries_europe['geometry'].apply(lambda x: x.representative_point().coords[:])
countries_europe['coords'] = [coords[0] for coords in countries_europe['coords']]

In [ ]:
ax = countries_europe.plot(figsize=(10,10))
for i, row in countries_europe.iterrows():
    plt.annotate(s=row['ISO_A3'], xy=row['coords'],
                 horizontalalignment='center')

In [ ]:
lat_min, lat_max = 30,72
lon_min, lon_max = -11,40


In [ ]:
boundaries = countries_europe['geometry']
boundaries

In [ ]:
from shapely.geometry import Point

In [ ]:
N = 100000
p_lat = np.random.uniform(low=lat_min,high=lat_max,size=N)
p_lon = np.random.uniform(low=lon_min,high=lon_max,size=N)
points = np.column_stack((p_lon,p_lat))
points_df = gpd.GeoDataFrame(geometry=[Point(lon,lat) for lon,lat in zip(p_lon,p_lat)])

In [ ]:
points_df

In [ ]:
plt.hist2d(points[:,0],points[:,1],bins=100);

In [ ]:
eu_boundary = countries_europe['geometry'].unary_union
print(type(eu_boundary))

In [ ]:
#in_eu = np.zeros((N,), dtype=bool)
#for i in range(N):
#    if eu_boundary.contains(Point(points[i])):
#        in_eu[i] = True

In [ ]:
in_eu = points_df.within(eu_boundary)

In [ ]:
points_df['in_eu'] = in_eu

In [ ]:
plt.figure(figsize=(10,10))
plt.hist2d(points[in_eu,0],points[in_eu,1],bins=100)
plt.show()

In [ ]:
points_df['country'] = np.nan
for i, country in countries_europe.iterrows():
    print(country['NAME_EN'])
    in_country = in_eu & points_df.loc[in_eu&points_df['country'].isnull(),'geometry'].within(country['geometry'])
    points_df.loc[in_country,'country'] = country['NAME_EN']

In [ ]:
points_df

In [ ]:
plt.figure(figsize=(10,10))
for country in points_df['country'].unique():
    in_country = (in_eu) & (points_df['country']==country)
    plt.scatter(points[in_country,0],points[in_country,1],s=1,label=country)
    
plt.legend(markerscale=5)
plt.show()

In [ ]:
points_df

In [ ]:
in_eu_not_in_country = in_eu&points_df['country'].isnull()

In [ ]:
in_eu_not_in_country.sum()

In [ ]:
in_eu.sum()